In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#import all vgg16 packages
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

In [ ]:
#construct an instance of ImageDataGenerator class
train_datagen = ImageDataGenerator(
    rescale = 1/255,
    horizontal_flip = True,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2
)
test_datagen = ImageDataGenerator(
    rescale = 1/255
)

In [ ]:
# instantiate VGG16 model
model = VGG16(include_top=False, input_shape=(224,224,3))

In [ ]:
# specify the layers loaded are not trainable
for layer in model.layers:
    layer.trainable = False

In [ ]:
# Flatten the output from second last fully connected layer in VGG
flatten_layer = layers.Flatten()(model.output)

#fully connected layer with 512 hidden units and ReLU activation
flattened_fc_layer = layers.Dense(512, activation='relu')(flatten_layer)

#The last fully connected sigmoid layer with 5 neurons
flattened_fc_softmax_layer = layers.Dense(5, activation='softmax')(flattened_fc_layer)


In [ ]:
#define a new model with base VGG model combined with the new layers just created
model = Model(inputs=model.inputs, outputs=flattened_fc_softmax_layer)

In [ ]:
#prepare iterators to loop through each and every image
training_iterator = train_datagen.flow_from_directory('dataset/flowers/train', batch_size=64, target_size=(224,224))
testing_iterator = test_datagen.flow_from_directory('dataset/flowers/test', batch_size=64, target_size=(224,224))


In [ ]:
#Compile and fit the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(training_iterator, validation_data = testing_iterator, epochs=4)

In [ ]:
import matplotlib.pyplot as plt

#plot loss vs epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('no of epochs')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [ ]:
#plot accuracy vs epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('no of epochs')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [ ]:
model.save('models/transfer_vgg16_flower.h5')